In [ ]:
import numpy as np
import pandas as pd
from encoding_loader import EncodingLoader 
from img_encoder import ModelType, ImageEncoder 
from nearest_neighbors import nearest_neighbors, display_progress, update_best
import time
import sys

In [ ]:
from nutritional_info import NutritionalInformation

In [ ]:
nut_info = NutritionalInformation()

Specify input file name

In [ ]:
input_img_filename = './test_img/lomein.jpg'

Select model type

In [ ]:
model_types = [ModelType.ResNet50, ModelType.ResNet101, ModelType.DenseNet121]
model_type = model_types[2]

Create image encoder and encoding loader

In [ ]:
enc_loader = EncodingLoader(model_type)
img_enc = ImageEncoder(model_type)

Run input image through the neural network in the image encoder to create a 2048-dimensional encoding of the input image. 

In [ ]:
enc = img_enc.encode_img(input_img_filename)[0]
print(enc.shape)

Select a distance metric for the nearest neighbor algorithm.

In [ ]:
metrics = ['cos', 'euclidean']
metric = metrics[0]
num_recipes = 3

Run nearest neighbor to find image encoding closest to the encoding of the input image. At the end of this cell, best_files is populated with filenames of the  𝑛𝑢𝑚_𝑟𝑒𝑐𝑖𝑝𝑒𝑠  images closes to our input image. The files are listed in order of decreasing similarity.

In [ ]:
enc_loader.reset_iter()
enc_matrix, index_to_filename = enc_loader.load_next_encoding()
best_files = [None] * num_recipes
best_sims = [None] * num_recipes
t = time.time()
i = 0
while True:
    display_progress(i)
    i += 1
    idx, similarity = nearest_neighbors(enc_matrix, enc, metric)
    best_files, best_sims = update_best(best_files, best_sims, similarity, index_to_filename[idx], metric)
    encoding = enc_loader.load_next_encoding()
    if encoding == None:
        break
    enc_matrix, index_to_filename = encoding
t = time.time() - t
display_progress(i)

print('\n')
for i in range(num_recipes):
    print('img ' + str(i + 1) + ': ' + best_files[i])
    print('similarity: ' + str(best_sims[i]))
print("Completed in " + str(t) + "s." )

Print the recipes and nutritional labels

In [ ]:
counter = 1
for f in best_files:
    print('\033[1m' + '\033[94m' + "Recipe " + str(counter) + '\033[0m')
    file = f[8:]
    info, nutr = nut_info.img_to_recipe(file)
    nut_info.display_recipe(nutr, info)
    print()
    counter += 1